In [ ]:
%load ./hello-world.ipynb

In [38]:
import dev.langchain4j.agent.tool.Tool;
import dev.langchain4j.model.output.structured.Description;
import dev.langchain4j.service.AiServices;
import dev.langchain4j.service.UserMessage;
import dev.langchain4j.service.SystemMessage;

import java.time.LocalDate;

## POJO as a return type
https://docs.langchain4j.dev/tutorials/ai-services#pojo-as-a-return-type

In [39]:
import dev.langchain4j.model.chat.request.ResponseFormatType;

ChatLanguageModel strictModel = OpenAiChatModel.builder()
    .apiKey(apiKey)
    .modelName("gpt-4o-mini")
    .logRequests(true)
    .logResponses(true)
    .responseFormat("json_schema") // JSONスキーマ出力を強制する
    .strictJsonSchema(true) // JSONスキーマ出力を強制する
    .build();

In [ ]:
record WeatherInfo(LocalDate date, String weather) {};
interface WeatherForecast {    
    @SystemMessage("天気の情報を取得")
    WeatherInfo getWetherInfo(String text);
}
WeatherForecast focast = AiServices.create(WeatherForecast.class, strictModel);
WeatherInfo info = focast.getWetherInfo("本日（2024/10/27）の天気は曇りです。");
info // WeatherInfo[date=2024-10-27, weather=曇り]

In [ ]:
@Description("住所")
record Address(String street, Integer streetNumber, String city) {}

record Person(
    String firstName,
    String lastName,
    LocalDate birthDate,
    Address address,
    @Description("どのような雰囲気の日に生まれたか") String feeling) {}

interface PersonExtractor {
    @UserMessage("以下のテキストから人物情報を抽出してください。抽出できない場合はnullを返してください。{{it}} ")
    Person extractPersonFrom(String text);
}

PersonExtractor personExtractor = AiServices.create(PersonExtractor.class, 
    // model);
    strictModel);

String text = """
            1968年、独立記念日の響きが薄れゆく中、穏やかな夕暮れの空の下にジョンという名の子供がやってきた。
            ドウ姓を名乗るこの新生児は、新たな旅の始まりを告げた。
            彼は、スプリングフィールドの中心部に位置する古風な通り、ウィスパリング・パインズ通り345番地に誕生した。
            """;

Person person = personExtractor.extractPersonFrom(text);
person // Person[firstName=ジョン, lastName=ドウ, birthDate=1968-07-04, address=Address[street=ウィスパリング・パインズ通り, streetNumber=345, city=スプリングフィールド], feeling=穏やかな夕暮れの空の下]